In [1]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import torch.nn.functional as F
from skopt import BayesSearchCV
from skopt.space import Real
from skopt import gp_minimize
import torch.nn as nn
import seaborn as sns
from scipy.optimize import minimize
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib as mpl
from skopt.utils import use_named_args
from tqdm import tqdm
import multiprocessing
import threading

In [2]:
# 设置GPU设备，如果没有可用的GPU，则使用CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(888888)

print(device)

cpu


## 基础参数

In [3]:
# 基础参数
epochs = 400000    # 训练代数
h = 100    # 画图网格密度
N = 100    # 内点配置点数
N1 = 0.01    # 边界点配置点数
n = 1000    # PDE数据点

## PINNs框架

In [4]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

## 损失函数

In [5]:
# Loss
loss = torch.nn.MSELoss()

# 递归求导
def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True,
                                   only_inputs=True, )[0]
    else:
        return gradients(gradients(u, x), x, order=order - 1)

In [6]:
X = torch.rand(n, 1, device=device, requires_grad=True)
Y = torch.rand(n, 1, device=device, requires_grad=True)

def point(n=100, device=device):
    # 随机生成 n 个点
    x = X
    y = Y
    return x, y

def l_ugeq(u, n=100):
    # u >= 0
    x, y = point(n, device)
    cond = torch.zeros_like(y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    llos = torch.relu(-uxy)
    ugeq_loss = 100 * torch.sum(llos)
    
    # 返回每个点的损失值
    return ugeq_loss, llos, x, y

def l_pde(u, n=100, device=device):
    # 等式项损失
    x, y = point(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    
    # 确保梯度计算返回每个点的值
    grad_x2 = gradients(uxy, x, 2)  # 对 x 计算二阶梯度
    grad_y2 = gradients(uxy, y, 2)  # 对 y 计算二阶梯度

    # 计算每个点的损失
    loss_value = -grad_x2 - grad_y2  # 假设这里是 PDE 中的计算
    loss_value = loss_value - cond   # 计算 PDE 的残差
    pde_loss = loss(-gradients(uxy, x, 2) - gradients(uxy, y, 2)- cond, torch.zeros_like(cond))

    # 返回每个点的损失值
    return pde_loss, loss_value, x, y


In [7]:
def l_JU(u, n=100, device=device):
    """
    计算每个点的 JU 损失值和整体的 JU 损失（通过蒙特卡罗积分近似）
    
    参数：
    - u: 神经网络
    - n: 采样点的数量
    - device: 设备（"cuda" 或 "cpu"）
    
    返回：
    - JU_loss: 总的目标函数值（通过积分近似）
    - llos_JU: 每个点的损失值（对应每个采样点）
    - x, y: 对应每个点的坐标
    """
    # 随机采样点
    x, y = point(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

    # 计算 u 和其梯度
    uxy = u(torch.cat([x, y], dim=1))
    grad_x = gradients(uxy, x, 1)
    grad_y = gradients(uxy, y, 1)
    
    # 每个点的损失值
    llos_JU = 0.5 * (grad_x**2 + grad_y**2) - cond * uxy  # 每个点对应的损失值
    p1 = 0.5 * (grad_x**2 + grad_y**2)
    p2 = cond * uxy

    # 使用蒙特卡罗积分近似整体损失
    JU_loss = llos_JU.mean()  # 取所有点的平均值作为积分的近似

    return JU_loss, llos_JU, x, y, p1, p2


In [8]:
# 边界函数构建
def l_boundary1(u):
    # 取点
    x = torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = torch.ones_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary2(u):
    # 取点
    y = torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = torch.ones_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary3(u):
    # 取点
    y = torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = torch.zeros_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary4(u):
    # 取点
    x = torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = torch.zeros_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

## 训练

In [ ]:
# Training
u = MLP().to(device)
opt = torch.optim.Adam(params=u.parameters(), lr=0.0001)

# 设置初始权重
weight_pde = 60000
weight_JU = 200
weight_boundary = 10000000
current_time = time.time()
maxerror = 99999999999

for epoch in range(epochs):
    opt.zero_grad()

    # # 手动调整学习率
    # if epoch == 100000:
    #     for param_group in opt.param_groups:
    #         param_group['lr'] *= 5  # 学习率放大 1/10 倍
    
    # 计算各个损失函数并获取点
    pde_loss, llos_pde, x_pde, y_pde = l_pde(u)
    JU_loss, llos_JU, x_JU, y_JU, p1, p2 = l_JU(u)
    ugeq_loss, llos_ugeq, x_ugeq, y_ugeq = l_ugeq(u)
    
    inside_loss = weight_pde * llos_pde + weight_JU * llos_JU + 6000 * llos_ugeq
    loss_boundary = l_boundary1(u) +  l_boundary2(u) + l_boundary3(u) + l_boundary4(u)
    
    # 计算加权总损失
    total_loss = weight_pde * pde_loss + weight_JU * JU_loss + 600 * ugeq_loss + weight_boundary * loss_boundary
    # total_loss = JU_loss
    # 每100次迭代记录一次损失
    if epoch % 100 == 0 and epoch != 0:
        print(f"pde:{pde_loss.item()},JU:{JU_loss.item()},boundary:{loss_boundary.item()}")
        print(p1.mean().item(), p2.mean().item())
        maxerror, current_time = record_loss_and_save_model(u, epoch, maxerror, current_time, h, device)
        
    # if epoch % 100000 == 0 and epoch != 0:
    #     X, Y = update_dataset(X, Y, inside_loss, x_pde, y_pde, top_k, num_new_points, bias, device)
    
    # 反向传播和优化
    total_loss.backward()
    opt.step()

